In [126]:
import sys
!{sys.executable} -m pip install -qU langchain-huggingface


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [127]:

story = """John is 15 years old and is on vacation with his wife Amira in Italy. Their daughter Anna can’t wait to visit the Eiffel Tower, but first they will go out to eat. John suggests they eat pizzas since Italy is famous for them, they will eat at restaurant Riccolo located in Florence.
Since Anna has a vitamin C deficiency she will order a pizza that contains lots of vitamin C, so she will get the pizza Bianca because that doesn't contain tomato's so this gives her more vitamin C than a pizza that does contain tomato's. John will get the classic Margherita pizza and Amira orders a pepperoni pizza.
They sit by the window of the small restaurant; the air filled with the smell of garlic and baking dough. Anna swings her legs impatiently under the table, still talking about the Eiffel Tower, while Amira flips through a guidebook about Florence.
15 minutes later the farmer named Leo from the restaurant brings their pizzas and they eat, and Anna says to John how cool it is that the waiter is from France."""


In [128]:
story = "Jack and his family are celebrating his birthday, Jack just turned 235 years old. Jack loves talking about himself and everything he has done in his life, many people would consider Jack a reserved person. This year, Jack’s family decided to visit the beautiful city of Quito, famous for its landmark La Virgen del Panecillo, which overlooks the city from a tall hill. Since the city doesn’t have mountainous terrain and is easy to walk through, Jack enjoys strolling through the streets without any trouble. It’s quite convenient, because Jack can’t walk very far due to his asthma. Luckily, Jack brought his medication, so whenever he’s out of breath, he can simply take his antihistamine pills."

In [129]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [130]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [131]:
# --- Local LLM for generation (Ollama)
from langchain_ollama import ChatOllama

# Initialize your local LLM (change model name if needed)
llm = ChatOllama(
    #model="phi3:mini",
    model = "llama3.1"
)

In [132]:
def scenario_splitter_llm(story, llm):
    story_to_scenarios_prompt = f"""
    You are an assistant that extracts logical or factual scenarios from a given story.

    A scenario is a small, self-contained statement or short passage that expresses one or more closely related facts, events, or claims that can later be checked for correctness or consistency using an ontology.
    ---

    ### Guidelines:
    - Each scenario should capture a complete idea, including all details that are logically connected (for example, cause–effect, contrast, or relationship).
    - If two or more statements are relevant to each other (e.g., one qualifies, contradicts, or explains the other), combine them into one scenario.
    - Preserve contextual and relational details — who, what, where, when, and why.
    - Avoid redundancy — do not restate the same information.
    - Keep each scenario brief but complete (usually one to three sentences).
    - Include implicit facts when they are important (e.g., “John’s wife Amira” implies John is married to Amira).
    - Ensure that every piece of relevant information from the story appears in at least one scenario.

    ---

    Example

    Story:
    “John is 15 years old and is on vacation with his wife Amira in Italy. Their daughter Anna can’t wait to visit the Eiffel Tower.”

    Extracted Scenarios:
    1. John is 15 years old and is married to Amira.
    2. John and Amira are on vacation in Italy.
    3. John and Amira have a daughter named Anna.
    4. Anna wants to visit the Eiffel Tower.

    ---

    Now extract the scenarios from the following story and number them clearly:

    {story}
    """

    # Query the LLM
    response = llm.invoke(story_to_scenarios_prompt)
    if hasattr(response, "content"):  # LangChain AIMessage
        output_text = response.content
    elif isinstance(response, dict) and "content" in response:
        output_text = response["content"]
    elif isinstance(response, str):
        output_text = response
    else:
        raise TypeError(f"Unexpected LLM response type: {type(response)}")

    # Extract list items using regex
    import re
    scenarios = re.findall(r'(?:\d+\.\s*)(.+)', output_text)
    scenarios = [s.strip() for s in scenarios if s.strip()]

    # If the LLM doesnt number them, fallback to line splitting
    if not scenarios:
        scenarios = [line.strip("-• \t") for line in output_text.splitlines() if line.strip()]

    return scenarios


In [133]:
scenarios = scenario_splitter_llm(story, llm)
scenarios

['Jack is 235 years old and celebrating his birthday with his family.',
 'Despite being considered a reserved person by many, Jack loves talking about himself.',
 'Jack has asthma that makes it difficult for him to walk far.',
 "Jack's family has decided to visit Quito, Ecuador, specifically to see the landmark La Virgen del Panecillo.",
 'The city of Quito is easy to navigate on foot due to its lack of mountainous terrain.']

In [134]:

from owlready2 import get_ontology, Thing, Restriction, And, Or, Not
from langchain_core.documents import Document

# Load your ontology
onto = get_ontology("./Ontology_Assignment.rdf").load()

#convert restriction into text
def restriction_to_text(restriction):

    #get name of property the restriction applies to
    #if property is None fallback to string, this shouldnt happen, but just incase 
    prop_name = restriction.property.name if restriction.property else str(restriction.property)

    # Check for "exists" attribute
    if hasattr(restriction, "some_values_from") and restriction.some_values_from: #check attribute and not empty
        #get name from target class or fallback to object itself
        target = getattr(restriction.some_values_from, "name", restriction.some_values_from)
        return f"must have some {prop_name} from {target}"
    
    # Check for "all" attribute
    elif hasattr(restriction, "all_values_from") and restriction.all_values_from:
        target = getattr(restriction.all_values_from, "name", restriction.all_values_from)
        return f"can only have {prop_name} from {target}"
    
    # Check for "≥" attribute
    elif hasattr(restriction, "min_cardinality") and restriction.min_cardinality is not None: #add is not None, because "0" would be handled as False
        return f"{prop_name} ≥ {restriction.min_cardinality}"
    
    # Check for "≤" attribute
    elif hasattr(restriction, "max_cardinality") and restriction.max_cardinality is not None: #add is not None, because "0" would be handled as False
        return f"{prop_name} ≤ {restriction.max_cardinality}"
    
    #Fallback for other types of restrictions
    else:
        return f"{prop_name} with unknown restriction {restriction}"

#recursive function that turns expression to text
def expression_to_text(expr):

    #Ontology class
    if hasattr(expr, "name"):
        return expr.name
    
    #constraint in ontology
    elif isinstance(expr, Restriction):
        return restriction_to_text(expr)
    
    #And expression
    elif isinstance(expr, And):
        return " and ".join(expression_to_text(c) for c in expr.Classes)
    
    #Or expression
    elif isinstance(expr, Or):
        return " or ".join(expression_to_text(c) for c in expr.Classes)
    
    #Not expr
    elif isinstance(expr, Not):
        return f"not {expression_to_text(expr.Class)}"

    #Fallback, return string
    else:
        return str(expr)

#turns ontology class to text
def class_to_text(cls):
    lines = [f"The class {cls.name} is defined as follows:"]
    
    # Superclasses
    for parent in cls.is_a:
        lines.append(f"- Subclass of {expression_to_text(parent)}")
    
    # Equivalent classes
    for eq in cls.equivalent_to:
        lines.append(f"- Equivalent to {expression_to_text(eq)}")
    
    return "\n".join(lines)

# turn object or data property to text
def property_to_text(prop):
    #starting text
    lines = [f"The property {prop.name} is defined as follows:"]
    
    #check for "domain" attribute and not empty
    if hasattr(prop, "domain") and prop.domain:
        #convert each class in domain to text
        domain_text = ", ".join(expression_to_text(d) for d in prop.domain)
        lines.append(f"- Domain: {domain_text}")
    
    #check for "range" attribute and not empty
    if hasattr(prop, "range") and prop.range:
        #convert each class in range to text
        range_text = ", ".join(expression_to_text(r) for r in prop.range)
        lines.append(f"- Range: {range_text}")
    
    # Only include characteristics if they exist
    if hasattr(prop, "is_functional") and prop.is_functional:
        lines.append("- This property is functional (at most one value).")
    if hasattr(prop, "is_inverse_functional") and prop.is_inverse_functional:
        lines.append("- This property is inverse functional (each value has at most one subject).")
    if hasattr(prop, "is_transitive") and prop.is_transitive:
        lines.append("- This property is transitive.")
    if hasattr(prop, "is_symmetric") and prop.is_symmetric:
        lines.append("- This property is symmetric.")
    if hasattr(prop, "is_asymmetric") and prop.is_asymmetric:
        lines.append("- This property is asymmetric.")
    if hasattr(prop, "is_reflexive") and prop.is_reflexive:
        lines.append("- This property is reflexive (every individual is related to itself).")
    if hasattr(prop, "is_irreflexive") and prop.is_irreflexive:
        lines.append("- This property is irreflexive (no individual is related to itself).")
    
    return "\n".join(lines)

# turn instances to text
def instance_to_text(instance):
    lines = [f"The instance {instance.name} is defined as follows:"]

    # Classes this instance belongs to
    types = [cls.name for cls in instance.is_a if hasattr(cls, "name")]
    if types:
        lines.append(f"- Belongs to class(es): {', '.join(types)}")

    # All properties (object and data) that this individual actually has
    for prop in instance.get_properties():
        try:
            values = getattr(instance, prop.python_name, [])
        except Exception:
            continue
        if values:
            # make iterable if not
            if not isinstance(values, (list, tuple, set)):
                values = [values]
            # check if each value has a name (object property) or is literal (data property)
            values_text = ", ".join(getattr(v, "name", str(v)) for v in values)
            lines.append(f"- {prop.name}: {values_text}")

    return "\n".join(lines)

# Collect all text for LLM
ontology_texts = []

# Add text for Classes
for cls in onto.classes():
    ontology_texts.append(class_to_text(cls))

# Add text for Object and datatype properties
for prop in onto.object_properties():
    ontology_texts.append(property_to_text(prop))
for prop in onto.data_properties():
    ontology_texts.append(property_to_text(prop))

# Add text for instances
for instance in onto.individuals():
    ontology_texts.append(instance_to_text(instance)) 

# Turn into LLM documents
docs = []
for text in ontology_texts:
    prompt = (
        "Convert the following ontology snippet into clear, human-readable English.\n"
        f"Snippet: {text}\n\n"
        "Do not add external information or commentary. Only describe what is present. Do not include any \"Here is a snippet\" to the output"
    )
    human_text = llm.invoke(prompt)
    print(human_text.content)
    docs.append(Document(page_content=human_text.content))

# Add to your vector store
vector_store.add_documents(docs)


Animals is a subclass of Thing.
The class Food is a subclass of Thing.
The class CookingStyles is a subclass of Thing.
The class Place is a subclass of the class Thing.
NoiseLevel is a subclass of Thing.
The class CharacterTrait is a subclass of Thing.
The class Behaviour is a subclass of Thing.
The class HealthCondition is a subclass of Health.
The class "Nutrient" is a subclass of "Food".
The class City is defined as follows:
It is a subclass of Place.
It must be located in some Country.
The class Health is a subclass of Thing.
FoodAllergies is defined as follows:

* It is a type of Allergy.
* It can come with symptoms that are typically associated with Rash.
* It can come with symptoms that are typically associated with Swollen Tongue.
* People who have FoodAllergies often take EPIpen as medication.
AnimalHairAllergies is defined as follows:

It is a type of AnimalAllergies.
It comes with Itching, and this is an unknown restriction.
It comes with Rash, and this is an unknown restric

['645be991-0664-45f1-baca-8360b2ed36b4',
 'c61617a3-8446-4ce9-bb6d-4ca77e9ed6d6',
 'a5816c5f-94fd-4cb9-a7d0-bdb79b9f623a',
 'ff159e95-8a07-46ff-855b-c9fdfd0b1583',
 '39fe0840-99df-4a45-b04d-1e003b0f695a',
 '048e0380-209e-4d86-850f-3db8065363bb',
 '17543df2-c3eb-4f34-bd14-b44c44b270d1',
 '709daf80-7041-4e75-8fcf-0942225bc981',
 '22cfe0f6-002a-453d-b1a7-95905432e1fa',
 '95e534a4-79a2-4907-9d6b-6037eee787dd',
 'faf0ad33-f7d4-43fc-acc3-5ca6dd3ea499',
 'e60802f8-e272-4fde-86ce-1dd42e5e6d3c',
 'e483727c-3a5d-4de0-b9a9-02b70493d789',
 '9ad422fe-cd0f-44d3-b8a2-65b7c4c41a02',
 '79c92cfe-230d-4987-afcd-def626a0e1c9',
 'c327b891-028f-418a-87c6-c711919a0dc4',
 '27600dc5-14f0-44c5-95ff-f1ab98667cb4',
 'e5cc6190-4264-4470-801f-53dad8ab8a58',
 '9ed77365-c4c3-4ebe-8cff-18cb9723c7be',
 '40dfb4ee-aa99-4c45-84ba-d89dde27ea24',
 '65c2744d-2c8c-4695-9479-efd13228c97f',
 '81ce56ca-f8db-4df7-953a-422e61c2ac11',
 'c77202ac-85a7-4b3d-9425-488d7cf47ffb',
 '64235f30-1c89-4634-a427-e35aa101bde4',
 '09f0b80a-f672-

In [135]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate.from_template("""
    You are a reasoning assistant that ensures each scenario in a story is logically consistent
    with both (a) the ontology information provided, and (b) the previously verified parts of the story.

    You will be given:
    1. Ontology context — definitions, class restrictions, domains/ranges, and relationships.
    2. The story so far — previously verified or corrected facts.
    3. A new scenario — the next statement to check.

    Your task:
    - Determine whether the new scenario is **consistent** with both the ontology and the story so far.
    - If it is **inconsistent**, rewrite it minimally so that it becomes consistent.
    - Use reasoning — e.g., check domain/range restrictions, disjoint classes, age constraints, or location hierarchies.
    - When fixing inconsistencies, preserve as much of the original story’s meaning as possible.
    - Ensure the corrected version still fits naturally within the ongoing story context.

    Output format (strictly follow this):
    Consistency: [Consistent / Inconsistent]
    Updated scenario: [Rewritten version of the scenario that fits ontology and story context]

    ---

    ### Example 1
    Ontology context:
    - Class Person: a human being.
    - Class Adult ⊆ Person: a person aged 18 or older.
    - Property hasSpouse domain=Adult, range=Adult.

    Story so far:
    John is 15 years old.

    Next scenario to verify:
    John is married to Amira.

    Expected reasoning:
    According to the ontology, only adults can have a spouse. John is 15, so this is inconsistent.

    Expected output:
    Consistency: Inconsistent
    Updated scenario: John is 23 years old and is married to Amira.

    ---

    ### Example 2
    Ontology context:
    - EiffelTower is locatedIn Paris.
    - Paris is locatedIn France.
    - Pisa is locatedIn Italy.

    Story so far:
    John and Amira are on vacation in Italy.

    Next scenario to verify:
    Anna says she wants to visit the Eiffel Tower.

    Expected reasoning:
    The Eiffel Tower is in France, but the story says the family is in Italy.

    Expected output:
    Consistency: Inconsistent
    Updated scenario: Anna says she wants to visit the Tower of Pisa.

    ---

    Now reason about the next scenario using the ontology and story so far.

    {context}

    Next scenario to verify:
    {question}
    """)


In [136]:
def incremental_reasoning(scenarios, vector_store, custom_prompt, llm, k=5):
    """
    Performs step-by-step reasoning on a sequence of scenarios, using accumulated
    story context and ontology-based retrieval for consistency checking and correction.
    """
    results = []
    accumulated_story = ""  # stores all accepted or corrected story so far

    for i, scenario in enumerate(scenarios, start=1):
        # 1. Retrieve ontology documents relevant to the current scenario
        retrieved_docs = vector_store.similarity_search(scenario, k=k)
        docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

        # 2. Construct reasoning context (ontology + accumulated story)
        full_context = (
            f"Ontology context:\n{docs_content}\n\n"
            f"Story so far:\n{accumulated_story.strip() or '(none so far)'}\n"
        )

        # 3. Build the complete prompt using the template
        rendered_prompt = custom_prompt.invoke({
            "question": scenario.strip(),
            "context": full_context.strip(),
        })

        # 4. Invoke the LLM
        answer = llm.invoke(rendered_prompt)
        answer_text = getattr(answer, "content", str(answer)).strip()

        # 5. Parse the model output (Consistency + Updated scenario)
        updated_scenario = scenario.strip()
        consistency_status = "Unknown"

        for line in answer_text.splitlines():
            line_lower = line.strip().lower()
            if line_lower.startswith("updated scenario:"):
                updated_scenario = line.split(":", 1)[1].strip()
            elif line_lower.startswith("consistency:"):
                consistency_status = line.split(":", 1)[1].strip()

        # 6. Ensure updated scenario has fallback if parsing failed
        if not updated_scenario:
            updated_scenario = scenario.strip()

        # 7. Update accumulated story (so the next scenario uses this corrected version)
        accumulated_story = (accumulated_story + "\n" + updated_scenario).strip()

        # 8. Store reasoning results
        results.append({
            "step": i,
            "original_scenario": scenario.strip(),
            "updated_scenario": updated_scenario,
            "consistency": consistency_status,
            "ontology_context": docs_content.strip(),
            "answer_raw": answer_text,
            "story_so_far": accumulated_story,
        })

    return results


In [137]:
results = incremental_reasoning(scenarios, vector_store, custom_prompt, llm, k=5)
results

[{'step': 1,
  'original_scenario': 'Jack is 235 years old and celebrating his birthday with his family.',
  'updated_scenario': "Jack is 67 years old and celebrating his birthday, but only as an Elder since he's reached that age milestone.",
  'consistency': 'Inconsistent',
  'ontology_context': "Leo belongs to the class of Adults and has the following characteristics:\nHe works as a Waiter at Riccolo.\nHis age is 23 years old.\n\nJohn belongs to the class of Adults and satisfies the following conditions:\n- John is married to Amira.\n- John has a child named Anna.\n- John's age is 34 years old.\n\nThe class Elder is defined as follows:\nElder is a subclass of Thing.\nElder is equivalent to Person and has an age that is at least 67 years old.\n\nMichael belongs to the class of Children and has one parent, Emily.\n\nA person is a thing.\nA person has at least one parent.\nThe age of a person is at most 110 years old.",
  'answer_raw': "Consistency: Inconsistent\nUpdated scenario: Jack 

In [138]:
prompt = f"Given the story: {story} and previously updated facts of the story: {results} change the original story so that it includes the original but is updated using the new facts"
new_story = llm.invoke(prompt)
new_story

AIMessage(content="Here's the updated story incorporating all the changes:\n\nJack is 67 years old and celebrating his birthday, but only as an Elder since he's reached that age milestone. Despite being considered a reserved person by many, Jack loves talking about himself - it's actually one of his favorite things to do! He's been known to share stories with his family for hours on end.\n\nThis year, Jack's family decided to visit Quito, Ecuador, specifically to see the famous LaVirgenDelPanecillo landmark. The city itself is mostly flat and has large pedestrian zones, making it relatively easy to walk around - although Jack does have some trouble walking far due to his asthma. Luckily, he always carries his medication with him, so whenever he gets out of breath, he can simply take an antihistamine pill to alleviate the symptoms.\n\nAs they stroll through Quito's streets, Jack points out various landmarks and shares stories about his own life experiences. Despite being 67 years old, h